In [1]:
pwd

'/home/physionet/project/code15_data_extraction'

**checking hea files**

In [ ]:
import os

# point this at the folder that contains your .hea files
FOLDER = '../data/code15False'

# find the first .hea
hea_file = None
for fn in os.listdir(FOLDER):
    if fn.endswith('.hea'):
        hea_file = os.path.join(FOLDER, fn)
        break

if not hea_file:
    print("No .hea files found in", FOLDER)
else:
    print("Inspecting:", hea_file)
    with open(hea_file, 'r') as f:
        for line in f:
            print(line.rstrip())

**copying ratiod samples to new folder**

In [4]:
import os
import shutil

INPUT_ROOT = '../data/code15False' 
OUTPUT_DIR = '../data/code15_unbalanced'   
os.makedirs(OUTPUT_DIR, exist_ok=True)

N_NEG = 17163
N_POS = 571

neg_count = 0
pos_count = 0

def read_chagas_label(hea_path):
    """
    Returns 1 for positive, 0 for negative, or None if no label found / unrecognized.
    """
    with open(hea_path, 'r') as f:
        for line in f:
            line = line.strip()
            if line.startswith('# Chagas label:'):
                val = line.split(':',1)[1].strip().lower()
                if val in ('true', '1', 'yes'):   # positive
                    return 1
                if val in ('false', '0', 'no'):   # negative
                    return 0
                return None
    return None

for root, dirs, files in os.walk(INPUT_ROOT):
    for fn in files:
        if not fn.endswith('.hea'):
            continue
        record = fn[:-4]
        hea = os.path.join(root, record + '.hea')
        dat = os.path.join(root, record + '.dat')
        if not os.path.exists(dat):
            continue

        label = read_chagas_label(hea)
        if label is None:
            continue

        if label == 0 and neg_count < N_NEG:
            neg_count += 1
        elif label == 1 and pos_count < N_POS:
            pos_count += 1
        else:
            continue

        # copy both into the same folder
        shutil.copy2(hea,  OUTPUT_DIR)
        shutil.copy2(dat,  OUTPUT_DIR)

        if neg_count >= N_NEG and pos_count >= N_POS:
            break
    else:
        continue
    break

print(f"Done: {neg_count} negatives + {pos_count} positives copied into {OUTPUT_DIR}")

Done: 17163 negatives + 571 positives copied into ../data/code15_unbalanced


**checking if prev code worked**

In [5]:
import os
from collections import Counter

OUTPUT_DIR = '../data/code15_unbalanced'  # ← your folder

def read_chagas_label(hea_path):
    with open(hea_path, 'r') as f:
        for line in f:
            if line.startswith('# Chagas label:'):
                val = line.split(':',1)[1].strip().lower()
                if val in ('true','1','yes'): return 1
                if val in ('false','0','no'): return 0
    return None

counts = Counter()
for fn in os.listdir(OUTPUT_DIR):
    if fn.endswith('.hea'):
        label = read_chagas_label(os.path.join(OUTPUT_DIR, fn))
        counts[label] += 1

print(f"Negatives (0): {counts[0]}")
print(f"Positives (1): {counts[1]}")
print(f"Unrecognized: {counts[None]}")

Negatives (0): 17163
Positives (1): 571
Unrecognized: 0
